# Detect claims to fact check in political debates

In this project you will implement various classifiers using both neural and feature based technqiues to detect which sentences in political debates should be fact checked.
Dataset from ClaimBuster: https://zenodo.org/record/3609356 
Evaluate your classifiers using the same metrics as http://ranger.uta.edu/~cli/pubs/2017/claimbuster-kdd17-hassan.pdf (Table 2)

Classification report from sklearn provides everything

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import *

# Loading the data

In [2]:
df = pd.read_csv("../data_preprocessing/data.csv")
df['date'] = pd.to_datetime(df['date'])
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23462 entries, 0 to 23461
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   index       23462 non-null  int64         
 1   date        23462 non-null  datetime64[ns]
 2   Text        23462 non-null  object        
 3   Clean_text  23462 non-null  object        
 4   Verdict     23462 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 916.6+ KB


# Train-test split

In [3]:
mask = df["date"].dt.year < 2012

X_train = df.loc[mask, "Clean_text"].values
y_train = df.loc[mask, "Verdict"].values

X_test = df.loc[~mask, "Clean_text"].values
y_test = df.loc[~mask, "Verdict"].values

# Data Preprocessing

In [4]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [5]:
# defining vocabulary
vocabulary = {}
sentences_len = []
for sentence in X_train:
    for term in sentence.split():
        vocabulary.setdefault(term, len(vocabulary))

In [6]:
# Defining vocabulary size
vocabulary_size = list(vocabulary.values())[-1] + 1

print(f"vocabulary is composed of {vocabulary_size} unique words")

vocabulary is composed of 10205 unique words


## One hot encoding representation

Encoding train data

In [7]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_encoded = tokenizer.texts_to_sequences(X_train)

Encodding Test data

In [8]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)

## Padding sequences

In [9]:
# finding max sentence length

vec_lengths = []
for i in X_train_encoded:
    vec_lengths.append(len(i))


max_length = np.unique(vec_lengths)[-1]


In [10]:
X_train_embedded=pad_sequences(X_train_encoded,padding='post',maxlen=max_length)
print(X_train_embedded)

[[  783   148     0 ...     0     0     0]
 [  130   110   771 ...     0     0     0]
 [  462  2841    30 ...     0     0     0]
 ...
 [    2  6525    43 ...     0     0     0]
 [ 1245    49   566 ...     0     0     0]
 [10205   264     1 ...     0     0     0]]


### For training

In [11]:
X_train_embedded.shape

(18118, 65)

### For testing

In [12]:
X_test_embedded=pad_sequences(X_test_encoded,padding='post',maxlen=max_length)
print(X_test_embedded.shape)

(5344, 65)


### For the labels

In [13]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
one_hot_encoder.fit(y_train.reshape(-1, 1))
y_encoded = one_hot_encoder.transform(y_train.reshape(-1, 1))

y_encoded.shape

(18118, 3)

In [14]:
y_encoded_test = one_hot_encoder.transform(y_test.reshape(-1,1))
y_encoded_test.shape

(5344, 3)

# Creating the model

In [115]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPool1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.backend import clear_session

In [116]:
clear_session()


### Defining plotting function

In [117]:
def plot_model_history(model_history=model.history):
    # plot loss during training
    plt.subplot(211)
    plt.title('Loss')
    plt.plot(model_history.history['loss'], label='train')
    plt.plot(model_history.history['val_loss'], label='test')
    plt.legend()
    # plot accuracy during training
    plt.subplot(212)
    plt.title('Accuracy')
    plt.plot(model_history.history['accuracy'], label='train')
    plt.plot(model_history.history['val_accuracy'], label='test')
    plt.legend()
    plt.subplots_adjust(hspace=0.5)
    plt.show()

## LSTM

In [118]:
model_lstm = Sequential()
model_lstm.add(Embedding(vocabulary_size+1, 81, input_length=max_length))
model_lstm.add(LSTM(100))
model_lstm.add(Dense(81, activation = "relu"))
model_lstm.add(Dense(3, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_lstm.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 65, 81)            826686    
                                                                 
 lstm (LSTM)                 (None, 100)               72800     
                                                                 
 dense (Dense)               (None, 81)                8181      
                                                                 
 dense_1 (Dense)             (None, 3)                 246       
                                                                 
Total params: 907,913
Trainable params: 907,913
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
model_lstm.fit(X_train_embedded,y_encoded, validation_split=0.2, epochs = 40)

Epoch 1/5
453/453 [==============================] - 14s 29ms/step - loss: 0.8384 - accuracy: 0.6730 - val_loss: 0.8874 - val_accuracy: 0.6360
Epoch 2/5
453/453 [==============================] - 13s 28ms/step - loss: 0.8333 - accuracy: 0.6731 - val_loss: 0.8871 - val_accuracy: 0.6360
Epoch 3/5
453/453 [==============================] - 13s 28ms/step - loss: 0.8325 - accuracy: 0.6731 - val_loss: 0.8977 - val_accuracy: 0.6360
Epoch 4/5
453/453 [==============================] - 13s 28ms/step - loss: 0.8325 - accuracy: 0.6731 - val_loss: 0.8868 - val_accuracy: 0.6360
Epoch 5/5
453/453 [==============================] - 13s 28ms/step - loss: 0.8324 - accuracy: 0.6730 - val_loss: 0.8886 - val_accuracy: 0.6360


In [ ]:
predictions = model_lstm.predict(X_test_embedded)
preds = one_hot_encoder.inverse_transform(predictions).reshape(-1,)
print(classification_report(y_test, preds, target_names=["NFS", "UFS", "CFS"]))


## Bidirectional LSTM 

In [53]:
model_bi = Sequential()
model_bi.add(Embedding(vocabulary_size+1, 97, input_length=max_length))
model_bi.add(Bidirectional(LSTM(100)))
model_bi.add(Dropout(0.5))
model_bi.add(Dense(97, activation = "relu"))
model_bi.add(Dense(3, activation='softmax'))
model_bi.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_bi.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 65, 97)            989982    
                                                                 
 bidirectional_5 (Bidirectio  (None, 220)              183040    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 220)               0         
                                                                 
 dense_12 (Dense)            (None, 97)                21437     
                                                                 
 dense_13 (Dense)            (None, 3)                 294       
                                                                 
Total params: 1,194,753
Trainable params: 1,194,753
Non-trainable params: 0
____________________________________________

In [54]:
model_bi.fit(X_train_embedded,y_encoded, validation_split=0.2, epochs = 3)

Epoch 1/3
453/453 [==============================] - 21s 41ms/step - loss: 0.6575 - accuracy: 0.7439 - val_loss: 0.6445 - val_accuracy: 0.7409
Epoch 2/3
453/453 [==============================] - 18s 40ms/step - loss: 0.4717 - accuracy: 0.8239 - val_loss: 0.6823 - val_accuracy: 0.7459
Epoch 3/3
453/453 [==============================] - 19s 41ms/step - loss: 0.3582 - accuracy: 0.8697 - val_loss: 0.7446 - val_accuracy: 0.7348


In [55]:
predictions = model_bi.predict(X_test_embedded)
preds = one_hot_encoder.inverse_transform(predictions).reshape(-1,)
print(classification_report(y_test, preds, target_names=["NFS", "UFS", "CFS"]))


              precision    recall  f1-score   support

         NFS       0.75      0.89      0.81      3296
         UFS       0.39      0.25      0.30       623
         CFS       0.64      0.46      0.54      1425

    accuracy                           0.70      5344
   macro avg       0.59      0.53      0.55      5344
weighted avg       0.68      0.70      0.68      5344



## Stacked Bi-LSTM

In [103]:
model_bi = Sequential()
model_bi.add(Embedding(vocabulary_size+1, 200, input_length=max_length))
model_bi.add(Dropout(0.2))
model_bi.add(Bidirectional(LSTM(100, return_sequences=True)))
model_bi.add(Bidirectional(LSTM(100)))
model_bi.add(Dropout(0.2))
model_bi.add(Dense(97, activation = "relu"))
model_bi.add(Dense(3, activation='softmax'))
model_bi.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_bi.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 65, 200)           2041200   
                                                                 
 dropout (Dropout)           (None, 65, 200)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 65, 200)          240800    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              240800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense (Dense)               (None, 97)                1

In [104]:
model_bi.fit(X_train_embedded,y_encoded, validation_split=0.2, epochs = 20)

Epoch 1/20
453/453 [==============================] - 52s 107ms/step - loss: 0.6497 - accuracy: 0.7473 - val_loss: 0.6650 - val_accuracy: 0.7312
Epoch 2/20
453/453 [==============================] - 47s 103ms/step - loss: 0.4797 - accuracy: 0.8170 - val_loss: 0.6686 - val_accuracy: 0.7409
Epoch 3/20
453/453 [==============================] - 46s 101ms/step - loss: 0.3636 - accuracy: 0.8662 - val_loss: 0.7582 - val_accuracy: 0.7332
Epoch 4/20
453/453 [==============================] - 47s 103ms/step - loss: 0.2689 - accuracy: 0.9029 - val_loss: 0.8505 - val_accuracy: 0.7243
Epoch 5/20
453/453 [==============================] - 46s 102ms/step - loss: 0.2005 - accuracy: 0.9295 - val_loss: 1.0763 - val_accuracy: 0.7144
Epoch 6/20
453/453 [==============================] - 50s 110ms/step - loss: 0.1513 - accuracy: 0.9477 - val_loss: 1.1367 - val_accuracy: 0.7058
Epoch 7/20
453/453 [==============================] - 50s 110ms/step - loss: 0.1222 - accuracy: 0.9574 - val_loss: 1.2871 - val_ac

In [101]:
predictions = model_bi.predict(X_test_embedded)
preds = one_hot_encoder.inverse_transform(predictions).reshape(-1,)
print(classification_report(y_test, preds, target_names=["NFS", "UFS", "CFS"]))


              precision    recall  f1-score   support

         NFS       0.75      0.91      0.82      3296
         UFS       0.36      0.28      0.32       623
         CFS       0.68      0.41      0.51      1425

    accuracy                           0.70      5344
   macro avg       0.60      0.53      0.55      5344
weighted avg       0.68      0.70      0.68      5344



## Convolutional Neural Network

In [85]:
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D
clear_session()


In [82]:
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocabulary_size+1, embedding_dim, input_length=max_length))
model.add(Conv1D(128, 10, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 65, 100)           1020600   
                                                                 
 conv1d_1 (Conv1D)           (None, 56, 300)           300300    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 300)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 64)                19264     
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                                 
Total params: 1,340,359
Trainable params: 1,340,359
Non-trainable params: 0
____________________________________________

In [83]:
model.fit(X_train_embedded,y_encoded, validation_split=0.2, epochs = 5)

Epoch 1/5
453/453 [==============================] - 8s 16ms/step - loss: 0.6677 - accuracy: 0.7371 - val_loss: 0.6508 - val_accuracy: 0.7343
Epoch 2/5
453/453 [==============================] - 7s 16ms/step - loss: 0.4292 - accuracy: 0.8388 - val_loss: 0.6923 - val_accuracy: 0.7337
Epoch 3/5
453/453 [==============================] - 7s 16ms/step - loss: 0.1954 - accuracy: 0.9307 - val_loss: 0.9051 - val_accuracy: 0.7150
Epoch 4/5
453/453 [==============================] - 7s 16ms/step - loss: 0.0902 - accuracy: 0.9690 - val_loss: 1.2466 - val_accuracy: 0.7183
Epoch 5/5
453/453 [==============================] - 7s 17ms/step - loss: 0.0441 - accuracy: 0.9868 - val_loss: 1.5655 - val_accuracy: 0.6981


In [84]:
predictions = model.predict(X_test_embedded)
preds = one_hot_encoder.inverse_transform(predictions).reshape(-1,)
print(classification_report(y_test, preds, target_names=["NFS", "UFS", "CFS"]))

              precision    recall  f1-score   support

         NFS       0.75      0.83      0.79      3296
         UFS       0.35      0.16      0.22       623
         CFS       0.53      0.53      0.53      1425

    accuracy                           0.67      5344
   macro avg       0.54      0.51      0.51      5344
weighted avg       0.65      0.67      0.65      5344



## Convolutional Neural network + LSTM

In [86]:
model_conv = Sequential()
model_conv.add(Embedding(vocabulary_size+1, 100, input_length=max_length))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(100, 8, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=10))
model_conv.add(LSTM(100))
model_conv.add(Dense(32, activation = "relu"))
model_conv.add(Dense(3, activation='softmax'))
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [87]:
model_conv.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 65, 100)           1020600   
                                                                 
 dropout (Dropout)           (None, 65, 100)           0         
                                                                 
 conv1d (Conv1D)             (None, 58, 100)           80100     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 5, 100)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 32)                3232      
                                                        

In [93]:
model_conv.fit(X_train_embedded,y_encoded, validation_split=0.4, epochs = 5)

Epoch 1/5
340/340 [==============================] - 5s 16ms/step - loss: 0.1060 - accuracy: 0.9617 - val_loss: 1.1266 - val_accuracy: 0.7007
Epoch 2/5
340/340 [==============================] - 5s 16ms/step - loss: 0.0644 - accuracy: 0.9780 - val_loss: 1.3545 - val_accuracy: 0.6995
Epoch 3/5
340/340 [==============================] - 6s 16ms/step - loss: 0.0511 - accuracy: 0.9823 - val_loss: 1.5313 - val_accuracy: 0.6980
Epoch 4/5
340/340 [==============================] - 6s 16ms/step - loss: 0.0430 - accuracy: 0.9856 - val_loss: 1.4808 - val_accuracy: 0.7118
Epoch 5/5
340/340 [==============================] - 5s 16ms/step - loss: 0.0335 - accuracy: 0.9894 - val_loss: 1.9262 - val_accuracy: 0.7003


In [92]:
predictions = model_conv.predict(X_test_embedded)
preds = one_hot_encoder.inverse_transform(predictions).reshape(-1,)
print(classification_report(y_test, preds, target_names=["NFS", "UFS", "CFS"]))


              precision    recall  f1-score   support

         NFS       0.74      0.89      0.81      3296
         UFS       0.26      0.24      0.25       623
         CFS       0.66      0.36      0.46      1425

    accuracy                           0.67      5344
   macro avg       0.55      0.50      0.51      5344
weighted avg       0.66      0.67      0.65      5344

